In [ ]:
import sys

sys.path.append("..")

from core.DataLoader import DataPreprocessor, get_load_config_from_yaml
from importlib import reload
import matplotlib.pyplot as plt
import os


if False:
    PLOTS_DIR = "plots/ToponiumInfusedTransformer/"
    MODEL_DIR = "models/mixed_model"
elif True:
    PLOTS_DIR = "plots/RegressionTransformer_nu_flows_neutrino_only/"
    MODEL_DIR = "../models/model_8874707/"
else:
    PLOTS_DIR = "plots/RegressionTransforsmer_/"
    MODEL_DIR = "../models/regression_transformer_old/"


CONFIG_PATH = "../config/workspace_config.yaml"

# Create directories
os.makedirs(PLOTS_DIR, exist_ok=True)

# Set plotting style
plt.rcParams.update({"font.size": 18})

# Load data
load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)
data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], event_numbers="odd"
)
PREDICTIONS_DIR = os.path.join(load_config.data_dir, "RegressionTransformer_nu_flows_neutrino_only", "odd_predictions/")
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

X, y = DataProcessor.get_data()
del DataProcessor

2026-01-29 10:22:29.076424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769678549.100261 1666126 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769678549.108049 1666126 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769678549.130484 1666126 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769678549.130509 1666126 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769678549.130511 1666126 computation_placer.cc:177] computation placer alr

In [2]:
import core.reconstruction.BaselineMethods as BaselineMethods
import core.evaluation.reconstruction_evaluator as Evaluation
from core.reconstruction import (
    GroundTruthReconstructor,
    PerfectAssignmentReconstructor,
    CompositeNeutrinoComponentReconstructor,
)

reload(Evaluation)
reload(BaselineMethods)
ground_truth_assigner = GroundTruthReconstructor(data_config, use_nu_flows=True)
perfect_assigner = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"PIL Transformer $\nu$"
)
perfect_assigner.load_model(f"{MODEL_DIR}/odd_model.keras")
composite_neutrino_predictor = CompositeNeutrinoComponentReconstructor(
    data_config, use_nu_flows=True, axis=(-1), name=r"PIL Transformer $\nu$"
)
composite_neutrino_predictor.load_model(f"{MODEL_DIR}/odd_model.keras")

prediction_manager = Evaluation.PredictionManager(
    [ground_truth_assigner, perfect_assigner,composite_neutrino_predictor], X, y
)


prediction_manager.save_predictions(PREDICTIONS_DIR)

I0000 00:00:1769678879.186169 1666126 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9699 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Model loaded from ../models/model_8874707//odd_model.keras
Training history loaded from ../models/model_8874707//odd_model_history.npz
Model loaded from ../models/model_8874707//odd_model.keras
Training history loaded from ../models/model_8874707//odd_model_history.npz


2026-01-29 10:28:06.404986: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1892989800 exceeds 10% of free system memory.
2026-01-29 10:28:09.714073: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1892989800 exceeds 10% of free system memory.
I0000 00:00:1769678892.932507 1671946 service.cc:152] XLA service 0x7ef8e80071f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1769678892.932534 1671946 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2026-01-29 10:28:13.009832: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2026-01-29 10:28:13.061904: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator keras_model_wrapper_1_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1769678893.455554 

Predictions saved for True Assignment + $\nu^2$-Flows.
Predictions saved for True Assignment + PIL Transformer $\nu$.
Predictions saved for PIL Transformer $\nu$ + $\nu^2$-Flows(z).
